Jonathan Gee

Running LDA on our wine dataset

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from nltk.stem.porter import *
stemmer = PorterStemmer()
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string
import nltk
import gensim
from gensim import corpora
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jonathan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /Users/jonathan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jonathan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jonathan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:

data = pd.read_csv("../data/winemag-data-130k-v2.csv")
data.dropna(inplace = True)
print(data.shape)
data.head()

(22387, 14)


,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
10,10,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
23,23,US,This wine from the Geneseo district offers aro...,Signature Selection,87,22.0,California,Paso Robles,Central Coast,Matt Kettmann,@mattkettmann,Bianchi 2011 Signature Selection Merlot (Paso ...,Merlot,Bianchi
25,25,US,Oak and earth intermingle around robust aromas...,King Ridge Vineyard,87,69.0,California,Sonoma Coast,Sonoma,Virginie Boone,@vboone,Castello di Amorosa 2011 King Ridge Vineyard P...,Pinot Noir,Castello di Amorosa
35,35,US,As with many of the Erath 2010 vineyard design...,Hyland,86,50.0,Oregon,McMinnville,Willamette Valley,Paul Gregutt,@paulgwine,Erath 2010 Hyland Pinot Noir (McMinnville),Pinot Noir,Erath


In [7]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [14]:
sample = data.iloc[0]['description']

print('original document: ')
words = []
for word in sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(sample))

original document: 
['Much', 'like', 'the', 'regular', 'bottling', 'from', '2012,', 'this', 'comes', 'across', 'as', 'rather', 'rough', 'and', 'tannic,', 'with', 'rustic,', 'earthy,', 'herbal', 'characteristics.', 'Nonetheless,', 'if', 'you', 'think', 'of', 'it', 'as', 'a', 'pleasantly', 'unfussy', 'country', 'wine,', "it's", 'a', 'good', 'companion', 'to', 'a', 'hearty', 'winter', 'stew.']


 tokenized and lemmatized document: 
['like', 'regular', 'bottl', 'come', 'rough', 'tannic', 'rustic', 'earthi', 'herbal', 'characterist', 'nonetheless', 'think', 'pleasantli', 'unfussi', 'countri', 'wine', 'good', 'companion', 'hearti', 'winter', 'stew']


In [21]:
#processed_desc = data.loc[0:100,['description']].applymap(preprocess)
#processed_desc.head()

#processed_desc = processed_desc.values

processed_desc = data['description'].map(preprocess)
processed_desc.head()

4     [like, regular, bottl, come, rough, tannic, ru...
10    [soft, suppl, plum, envelop, oaki, structur, c...
23    [wine, geneseo, district, offer, aroma, sour, ...
25    [earth, intermingl, robust, aroma, forest, flo...
35    [erath, vineyard, design, strongli, herbal, no...
Name: description, dtype: object

In [22]:
dictionary = gensim.corpora.Dictionary(processed_desc)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 bottl
1 characterist
2 come
3 companion
4 countri
5 earthi
6 good
7 hearti
8 herbal
9 like
10 nonetheless


In [23]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [24]:
bow_corpus = [dictionary.doc2bow(desc) for desc in processed_desc]
bow_corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1)]

In [25]:
bow_doc_4310 = bow_corpus[0]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 0 ("bottl") appears 1 time.
Word 1 ("characterist") appears 1 time.
Word 2 ("come") appears 1 time.
Word 3 ("companion") appears 1 time.
Word 4 ("countri") appears 1 time.
Word 5 ("earthi") appears 1 time.
Word 6 ("good") appears 1 time.
Word 7 ("hearti") appears 1 time.
Word 8 ("herbal") appears 1 time.
Word 9 ("like") appears 1 time.
Word 10 ("nonetheless") appears 1 time.
Word 11 ("pleasantli") appears 1 time.
Word 12 ("regular") appears 1 time.
Word 13 ("rough") appears 1 time.
Word 14 ("rustic") appears 1 time.
Word 15 ("stew") appears 1 time.
Word 16 ("tannic") appears 1 time.
Word 17 ("think") appears 1 time.
Word 18 ("unfussi") appears 1 time.


In [26]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.09966859660975297),
 (1, 0.22647119663306559),
 (2, 0.1297317054310132),
 (3, 0.27146076727364016),
 (4, 0.3098664608133152),
 (5, 0.14124919658276547),
 (6, 0.14079933673089798),
 (7, 0.21328960936752547),
 (8, 0.15836129686412348),
 (9, 0.1272778066794725),
 (10, 0.2796659529458348),
 (11, 0.2696122989249046),
 (12, 0.2819582076357378),
 (13, 0.2588373331204153),
 (14, 0.22431687021128732),
 (15, 0.245830588670773),
 (16, 0.15122010109191425),
 (17, 0.28690187493381614),
 (18, 0.3352019092481413)]


In [27]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [28]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.043*"fruit" + 0.026*"aroma" + 0.021*"cherri" + 0.019*"spice" + 0.018*"finish" + 0.017*"herb" + 0.013*"palat" + 0.013*"tannin" + 0.011*"barrel" + 0.011*"note"
Topic: 1 
Words: 0.018*"aroma" + 0.018*"fruit" + 0.017*"lemon" + 0.016*"light" + 0.015*"blend" + 0.014*"palat" + 0.014*"fresh" + 0.013*"acid" + 0.012*"appl" + 0.012*"finish"
Topic: 2 
Words: 0.022*"fruit" + 0.012*"vineyard" + 0.009*"rich" + 0.009*"acid" + 0.009*"sugar" + 0.009*"cherri" + 0.008*"finish" + 0.008*"soft" + 0.007*"barrel" + 0.007*"drink"
Topic: 3 
Words: 0.052*"cabernet" + 0.031*"sauvignon" + 0.031*"blend" + 0.027*"merlot" + 0.024*"fruit" + 0.019*"franc" + 0.019*"petit" + 0.019*"tannin" + 0.015*"finish" + 0.014*"verdot"
Topic: 4 
Words: 0.022*"finish" + 0.020*"tannin" + 0.019*"fruit" + 0.019*"black" + 0.015*"cherri" + 0.011*"soft" + 0.011*"palat" + 0.010*"vineyard" + 0.009*"bodi" + 0.009*"ripe"
Topic: 5 
Words: 0.028*"fruit" + 0.027*"black" + 0.020*"dark" + 0.018*"cherri" + 0.015*"tannin" + 0.014*"ch

In [29]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))


Topic: 0 Word: 0.007*"appl" + 0.006*"bodi" + 0.006*"aroma" + 0.006*"pear" + 0.006*"light" + 0.005*"fruit" + 0.005*"acid" + 0.005*"finish" + 0.005*"note" + 0.005*"lemon"
Topic: 1 Word: 0.006*"fruit" + 0.006*"finish" + 0.005*"cherri" + 0.005*"strawberri" + 0.005*"light" + 0.005*"acid" + 0.005*"bodi" + 0.005*"drink" + 0.005*"vineyard" + 0.005*"tannin"
Topic: 2 Word: 0.006*"cherri" + 0.006*"fruit" + 0.006*"black" + 0.005*"vineyard" + 0.005*"finish" + 0.005*"tannin" + 0.005*"acid" + 0.004*"dark" + 0.004*"spice" + 0.004*"palat"
Topic: 3 Word: 0.009*"black" + 0.007*"cherri" + 0.006*"tannin" + 0.005*"pepper" + 0.005*"bodi" + 0.005*"fruit" + 0.005*"blackberri" + 0.005*"nose" + 0.005*"palat" + 0.005*"dri"
Topic: 4 Word: 0.008*"black" + 0.007*"fruit" + 0.006*"aroma" + 0.006*"cherri" + 0.006*"dark" + 0.006*"palat" + 0.006*"note" + 0.005*"spice" + 0.005*"blend" + 0.005*"nose"
Topic: 5 Word: 0.018*"cabernet" + 0.012*"merlot" + 0.011*"sauvignon" + 0.010*"franc" + 0.009*"blend" + 0.009*"herb" + 0.008*

In [30]:
for index, score in sorted(lda_model[bow_corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6563329100608826	 
Topic: 0.028*"fruit" + 0.027*"black" + 0.020*"dark" + 0.018*"cherri" + 0.015*"tannin" + 0.014*"chocol" + 0.011*"time" + 0.011*"bottl" + 0.010*"vineyard" + 0.009*"age"

Score: 0.3036459982395172	 
Topic: 0.022*"finish" + 0.020*"tannin" + 0.019*"fruit" + 0.019*"black" + 0.015*"cherri" + 0.011*"soft" + 0.011*"palat" + 0.010*"vineyard" + 0.009*"bodi" + 0.009*"ripe"


In [31]:
for index, score in sorted(lda_model_tfidf[bow_corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.7075772285461426	 
Topic: 0.008*"black" + 0.007*"fruit" + 0.006*"aroma" + 0.006*"cherri" + 0.006*"dark" + 0.006*"palat" + 0.006*"note" + 0.005*"spice" + 0.005*"blend" + 0.005*"nose"

Score: 0.2523968815803528	 
Topic: 0.006*"fruit" + 0.006*"finish" + 0.005*"cherri" + 0.005*"strawberri" + 0.005*"light" + 0.005*"acid" + 0.005*"bodi" + 0.005*"drink" + 0.005*"vineyard" + 0.005*"tannin"


In [32]:
unseen_document = 'This wine comes from the northernmost of the Grand Crus in Gevrey-Chambertin. Immense red fruits are as immense as the tannins, both laced with intense, juicy acidity. The wines richness will be enhanced as it ages. Drink from 2027'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5124780535697937	 Topic: 0.022*"fruit" + 0.012*"vineyard" + 0.009*"rich" + 0.009*"acid" + 0.009*"sugar"
Score: 0.43749716877937317	 Topic: 0.030*"black" + 0.027*"palat" + 0.026*"nose" + 0.021*"cherri" + 0.019*"fruit"
